In [1]:
import math
import pickle

import tqdm
import mlflow
import numpy as np
from scipy import stats
from scipy.optimize import fsolve

from utils import *

1. Прочитайте и проанализируйте данные, выберите турниры, в которых есть данные о
составах команд и повопросных результатах (поле mask в results.pkl). Для унификации
предлагаю:

In [2]:
with open('chgk/players.pkl', 'rb') as f:
    players = pickle.load(f)
    
with open('chgk/results.pkl', 'rb') as f:
    results = pickle.load(f)
    
with open('chgk/tournaments.pkl', 'rb') as f:
    tournaments = pickle.load(f)

In [3]:
train_ids = get_ids_by_date([2019], tournaments, results)
test_ids = get_ids_by_date([2020], tournaments, results)

In [4]:
print(f"Количество турниров для обучения {len(train_ids)}, количество турниров для теста {len(test_ids)}")

Количество турниров для обучения 659, количество турниров для теста 165


In [5]:
# Функции для рассчета коэффициента корреляции Спирмена и Кендалла

def spearman(pred_ratings, actual_ratings):
    corrs = []
    for pr, tr in zip(pred_ratings, actual_ratings):
        corrs.append(stats.spearmanr(pr, tr).correlation)
    index = ~np.isnan(corrs) * 1
    corrs = np.array(corrs)[index]
    corr_coef = np.mean(corrs)
    print('Spearman', corr_coef)
    return corr_coef
    
def kendall(pred_ratings, actual_ratings):
    corrs = []
    for pr, tr in zip(pred_ratings, actual_ratings):
        corrs.append(stats.kendalltau(pr, tr).correlation)
    index = ~np.isnan(corrs) * 1
    corrs = np.array(corrs)[index]
    corr_coef = np.mean(corrs)
    print('Kendall', corr_coef)
    return corr_coef
    
def rating_to_score(ratings):
    positions = []
    sorted_r = sorted(ratings)
    for rating in ratings:
        positions.append(sorted_r.index(rating) + 1)
    return positions

def calc_correlations(ids, results, players_mean_mu):
    pred_ratings = []
    actual_ratings = []
    for id_ in ids:
        tournament = results[id_]
        t_pred_ratings = []
        t_actual_ratings = []
        for team in tournament:
            scores = []
            for member in team['teamMembers']:
                member_id = member['player']['id']
                if member_id in players_mean_mu:
                    scores.append(players_mean_mu[member_id])
            t_actual_ratings.append(team['position'])
            if len(scores):
                index = ~np.isnan(scores)
                scores = list(np.array(scores)[index])
                if len(scores) == 0:
                    t_pred_ratings.append(0)
                else:
                    prob = -np.mean(scores)
                    t_pred_ratings.append(prob)
            else:
                t_pred_ratings.append(0)
        pred_ratings.append(rating_to_score(t_pred_ratings))
        actual_ratings.append(t_actual_ratings)
    
    s_corr = spearman(pred_ratings, actual_ratings)
    k_corr = kendall(pred_ratings, actual_ratings)
    return s_corr, k_corr

2. Постройте baseline-модель на основе линейной или логистической регрессии, которая
будет обучать рейтинг-лист игроков. 

Проанализировав Ваши замечания и подсказки, я решил (на свой риск и страх) сделать бэйзлайн модель не на основе логистической, либо линейной регрессии, а сделать итерационный алгоритм, правда ничем неподкрепленный, но мне он показался "напрашивающимся".

Так как ответ игрока на какой-либо вопрос $x$ это "верно" либо "неверно", я сделал предположение, что сила какого-либо игрока это параметр $mu_k$ распределения Бернулли, и он зависит от сложности турнира $w_t$:


Вероятность игрока k правильно ответит на вопрос турнира t:

$p(x=1|k, t) = mu_{kt}$

По методу максимального правдоподобия, параметр распределения Бернулли равен отношению количества правильных ответов к количеству всех вопросов: 
        
$mu_{k}$ = $\frac{1}{N}$$\sum_{i=1}^{N} I(x_i)$

где $I(x_i)$ индикатор, равен 1 если игрок ответил правильно но вопрос $x_i$, и 0 в обратном случае

Но проблема в том, что каждый турнир имеет разную сложность и вот так просто найти силу игрока не получится.
Поэтому я ввел параметр $w_t$ - сложность турнира

$mu_{mle}$ для игрока k на турнира t равно ($N_t$ - количество вопросов на турнире t):
        
1. $mu_{kt}$ = $\frac{1}{N_t}$$\sum_{i=1}^{N_t} I(x_i)$

А полная вероятность равна (T количество турниров игрока k):
        
2. $mu_{k}$ = $\sum_{t=1}^{T} mu_{kt} * w_{t}$

Возникает вопрос чему равен $w_t$. Например в футболе, для меня лично, сложность турнира определяется количеством сильных команд. Например, лига чемпионов выше уровнем чем лига европы, потому что в 1-турнире участвуют больше грандов.

Поэтому $w_t$ определим как среднюю силу всех игроков, участвовавших на турнире t ($N_p$ - количество участников на турнире t):

3. $w_t$ = $\frac{1}{N_p}$$\sum_{k=1}^{N_p} mu_{k}$

И так получилось следующее:
1. Инициализируем $w_t$ случайными значениями от 0 до 1
2. Находим $mu_{k}$ по формуле 2
3. Рассчитываем $w_t$ по формуле 3
4. Повторяем 2-3 шаги до схождения

In [6]:
class BaselineModel:
    def __init__(self, train_ids, results):
        self.player_appereances = get_players_appereances(train_ids, results)
        self.players_mean_mu = create_players_dict(train_ids, results, True)
        self.mu_vals = create_nested_dict(train_ids, results)
        
        """Инициализируем сложность турниров случайными значениями"""
        comp_probs = np.random.uniform(low=1.0, high=2.0, size=len(train_ids))
        self.comp_probs = {id_: prob for id_, prob in zip(train_ids, comp_probs)}
        self.consider_questions_num = False
        
        self.train_ids = train_ids
        self.results = results
        
    def calc_players_strength_for_competition(self, comp_id, weight, eps=1e-5):
        """Расчитываем формулу 2 для каждого игрока на турнире comp_id"""
        for team in self.results[comp_id]:
            team_id = team['team']['id']
            mask = team['mask']
            mask = to_int(mask)
            if mask.shape[0] > 0 and len(team['teamMembers']) != 0:
                for i, player in enumerate(team['teamMembers']):
                    player_id = player['player']['id']
                    new_mu = np.mean(mask)
                    if self.consider_questions_num:
                        penalty = np.sqrt(np.log10(self.player_appereances[player_id] + 1))
                        self.players_mu[player_id].append(new_mu * weight * penalty)                
                    else:
                        self.players_mu[player_id].append(new_mu * weight)
                        
    def get_comp_prob(self, comp_id):
        """Рассчитываем формулу 3 для турнира comp_id"""
        comp = self.mu_vals[comp_id]
        scores = []
        max_mu = 1e-5
        for team in comp:
            for player in comp[team]:
                comp[team][player] = np.mean(self.players_mu[player])
                scores.append(comp[team][player])
                if comp[team][player] > max_mu:
                    max_mu = comp[team][player]
        if len(scores):
            return np.mean(scores), max_mu
        return 1e-5, max_mu
    
    def get_comp_probs(self):
        """Рассчитываем формулу 3 для всех турниров и нормализуем сложность турнира и силу игроков"""
        gl_max_mu = 1e-5
        for comp_id in tqdm.tqdm(self.mu_vals):
            prob, max_mu = self.get_comp_prob(comp_id)
            self.comp_probs[comp_id] = prob
            if max_mu > gl_max_mu:
                gl_max_mu = max_mu
        norm = np.array(list(self.comp_probs.values())).sum()
        for comp_id in self.mu_vals:
            self.comp_probs[comp_id] = self.comp_probs[comp_id] / (norm + 1e-4)
        for player_id in self.players_mu:
            self.players_mu[player_id] = list(np.array(self.players_mu[player_id]) / gl_max_mu)
            self.players_mean_mu[player_id] = np.mean(self.players_mu[player_id])
    
    def iteration(self):
        """Запускаем итерацию 2-3 пунктов"""
        self.players_mu = create_players_dict(self.train_ids, self.results)
        for comp_id in self.comp_probs:
            self.calc_players_strength_for_competition(comp_id, self.comp_probs[comp_id])
        self.get_comp_probs()

Качество рейтинг-системы оценивается качеством предсказаний результатов турниров.
Но сами повопросные результаты наши модели предсказывать вряд ли смогут, ведь
неизвестно, насколько сложными окажутся вопросы в будущих турнирах; да и не нужны
эти предсказания сами по себе. Поэтому:

- предложите способ предсказать результаты нового турнира с известными
составами, но неизвестными вопросами, в виде ранжирования команд;
- в качестве метрики качества на тестовом наборе давайте считать ранговые
корреляции Спирмена и Кендалла (их можно взять в пакете scipy) между
реальным ранжированием в результатах турнира и предсказанным моделью,
усреднённые по тестовому множеству турниров 1 .

Обучим бэйзлайн модель

In [7]:
model = BaselineModel(train_ids, results)
experiment_id = mlflow.set_experiment("baseline-model")
with mlflow.start_run(experiment_id=experiment_id):
    for i in range(30):
        model.iteration()
        print('Iteration', i + 1)
        s_corr, k_corr = calc_correlations(test_ids, results, model.players_mean_mu)
        mlflow.log_metric("Spearman", s_corr, i + 1)
        mlflow.log_metric("Kendall", k_corr, i + 1)
        print('\n')

100%|██████████| 659/659 [00:04<00:00, 160.94it/s]


Iteration 1
Spearman 0.7130498064981999
Kendall 0.538863941223462




100%|██████████| 659/659 [00:04<00:00, 155.66it/s]


Iteration 2
Spearman 0.7627896321484713
Kendall 0.5871775638842168




100%|██████████| 659/659 [00:04<00:00, 156.29it/s]


...

Iteration 28
Spearman 0.7957590406894631
Kendall 0.6188709517282583




100%|██████████| 659/659 [00:04<00:00, 160.22it/s]


Iteration 29
Spearman 0.7957590406894631
Kendall 0.6188709517282583




100%|██████████| 659/659 [00:03<00:00, 166.21it/s]


Iteration 30
Spearman 0.7957590406894631
Kendall 0.6188709517282583




Видим, что после 22-итерации метрики сошлись и не меняются в последующих итерациях. Взглянем на топ-100 игроков (имя знатока, его сила mu и количество сыгранных вопросов):

In [8]:
top_players = get_top_players(model.players_mean_mu, 100, players, model.player_appereances)
for i, player in enumerate(top_players):
    print(i + 1, player[0], player[1], player[2])

1 Роман Петров 1.0 30
2 Игорь Петров 1.0 30
3 Андриан Иоаннидис 1.0 30
4 Дмитрий Луконин 1.0 30
5 Анатолий Королихин 1.0 30
6 Евгений Чернецкий 0.8888888888888888 30
7 Александр Усов 0.8888888888888888 30
8 Дамир Фёдоров 0.8888888888888888 30
9 Василий Марков 0.8888888888888888 30
10 Константин Карпов 0.611111111111111 30
11 Иван Карпов 0.611111111111111 30
12 Виктория Кулёва 0.611111111111111 30
13 Яна Кулёва 0.611111111111111 30
14 Юлия Крюкова 0.04636495022484732 36
15 Наталья Артемьева 0.04636495022484732 36
16 Екатерина Горелова 0.04636495022484732 36
17 Антон Калинин 0.04491604553032085 36
18 Ольга Остросаблина 0.04281642817696037 36
19 Валентина Подюкова 0.04215803143976931 36
20 Махбуба Мамаджанова 0.04201823614126789 36
21 Дарина Калнина 0.04056933144674141 36
22 Артём Улюкин 0.04056933144674141 36
23 Мария Каменских 0.039914776748728883 72
24 Сергей Пашкевич 0.03918952132896044 36
25 Анна Щеголькова 0.03767152205768845 36
26 Матвей Конюхов 0.03767152205768845 36
27 Иван Конюх

В топе рейтинга игроков действительно ноунеймы, сыгравшие мало вопросов. Также в рейтинге есть игроки и с большим количеством сыгранных вопросов, и вроде бы все они действительно топы. А теперь взглянем на топ-100 турниров по сложности

In [9]:
top_comps = get_top_comps(model.comp_probs, 100, tournaments)
for i, comp in enumerate(top_comps):
    print(i + 1, comp[0], comp[1])

1 Чемпионат Кипра среди школьников 0.07298699088929253
2 Чемпионат Мира. Финал. Группа А 0.002539260697966148
3 Чемпионат Мира. Этап 2. Группа А 0.00251638038937184
4 Чемпионат Мира. Этап 3. Группа А 0.0025124006775994793
5 Чемпионат Мира. Этап 1. Группа А 0.0023271362653579045
6 Чемпионат Санкт-Петербурга. Высшая лига 0.002150883339162054
7 Чемпионат России 0.002122722194604122
8 Чемпионат Мира. Финал. Группа В 0.0020940010394881535
9 Чемпионат Мира. Этап 3. Группа В 0.0020890240764819827
10 Славянка 0.0020819929152066543
11 Чемпионат Мира. Этап 2. Группа В 0.002073090945550594
12 Весна в ЛЭТИ 0.0020075890969397667
13 Мемориал Дмитрия Коноваленко 0.0019999925940831077
14 Второй тематический турнир имени Джоуи Триббиани 0.0019979020298069874
15 Мемориал Дмитрия Мотрича 0.001991505169463686
16 Год театра 0.0019864888379788
17 Чемпионат Мира. Этап 1. Группа В 0.0019707882354188765
18 Регулярный чемпионат МГУ. Высшая лига. Третий игровой день 0.00196799242383378
19 Знатокиада. Всеобщий От

На первом месте чемпионат среди школьников, видимо те ноунеймы и подняли рейтинг этого турнира, далее идут разные этапы чемпионата мира и чемпионат России, вроде более-менее правильно

Теперь главное: ЧГК — это всё-таки командная игра. Поэтому:
- предложите способ учитывать то, что на вопрос отвечают сразу несколько
игроков; скорее всего, понадобятся скрытые переменные; не стесняйтесь делать
упрощающие предположения, но теперь переменные “игрок X ответил на вопрос
Y” при условии данных должны стать зависимыми для игроков одной и той же
команды;
- разработайте EM-схему для обучения этой модели, реализуйте её в коде;
- обучите несколько итераций, убедитесь, что целевые метрики со временем
растут (скорее всего, ненамного, но расти должны), выберите лучшую модель,
используя целевые метрики.

Рассмотрим правдоподобие результата $X$ какой-либо команды $m$ на каком-либо турнире $t$ если в команде только один игрок:

$p(X_{tm}|mu_m, t) = \prod_{i=1}^{N} mu_m^{x_i} * (1 - mu_m) ^ {1 - x_i}$

Но в командах обычно больше чем один игрок. Нам неизвестно кто из игроков ответил на определенный вопрос. Вообще говоря бывают разные случаи: бывает все игроки знают ответ, бывает несколько игроков знают и отвечают правильно, бывает несколько знатоков имеют правильную версию, но отвечает игрок с неправильной версией. Смоделировать эти ситуации сложно. Я решил упростить задачу и сделать предположение, что если команда ответила неправильно, то ни у кого не было правильной версии. Дальше я сделал предположение, что на отвеченный вопрос знал ответ только один знаток. Поэтому у меня получилась некая класстеризация вопросов по игрокам. Давайте рассмотрим правдоподобие результата (K количество игроков в команде m, $pi_k$ - равно 1 если k-ый игрок ответил на вопрос, и 0 если нет):

$p(X_{tm}|mu_{m1}, mu_{m2} ... mu_{mk}) = \prod_{i=1}^{N} (\sum_{k=1}^{K} pi_k * mu_{mk}^{x_i} * (1 - mu_{mk}) ^ {1 - x_i})$

$\sum_{k=1}^{K} pi_k = 1$

Рассмотрим логарифм правдоподобия:

$log(p(X_{tm}|mu_{m1}, mu_{m2} ... mu_{mk}, pi)) = \sum_{i=1}^{N} (log(\sum_{k=1}^{K} pi_{mk} * mu_{mk}^{x_i} * (1 - mu_{mk}) ^ {1 - x_i}))$

И как мы видим максимизировать это правдоподобие сложно, поэтому внесем скрытые переменные $z$:

$log(p(X_{tm}|mu_{m1}, mu_{m2} ... mu_{mk}, z)) = \sum_{i=1}^{N} (log(\prod_{k=1}^{K} (mu_{mk}^{x_i} * (1 - mu_{mk}) ^ {1 - x_i})^ {z_{mk}}))$

$\sum_{k=1}^{K} z_k = 1$

Теперь распишем правдободобие для всех данных (все турниры T, все команды M, все вопросы N):

$log(X|everything) = \sum_{t=1}^{T}\sum_{m=1}^{M}\sum_{i=1}^{N} (log(\prod_{k=1}^{K} (mu_{mk}^{x_i} * (1 - mu_{mk}) ^ {1 - x_i})^ {z_{mk}}))$

Найти ожидание $z_{mk}$ можно по формуле:

1. $E[z_{mk}] = \frac{pi_{mk} * p(X_{tm}|mu_{mk}, t)}{\sum_{j=1}^{K}pi_{mj} * p(X_{tm}|mu_{mj}, t)} $

Опять же тут у всех турниров разные уровни, поэтому я ввел следующее равенство, вероятность игрока k ответит на вопрос турнира t равно (p_k - сила игрока, w_t - сложность вопроса):

2. $p(x=1|k, t) = \frac{p_k}{p_k + w_t} = mu_{kt}$


Теперь приступим к EM-алгоритму:

- на шаге expectation найдем все $E[z_{mkt}]$
- на шаге максимизации нужно найти значения $pi_{mkt}$, $w_t$, $p_k$. $mu_{kt}$ найдем по формуле 2


9. $pi_{kt} = \frac{1}{N}\sum_{n=1}^{N}E[z_{nkt}]$   (формулы 9.60 из Bishop)

А для нахождения $p_k$ и $w_t$ распишем часть правдободобия где присутсвуют эти параметры:

$l = \sum_{t=1}^{T}\sum_{n=1}^{N_t}z_{nt} * log(\frac{p}{p + w_t}^{x_n} * (1 - \frac{p}{p + w_t})^{1 - x_n})$

$l = \sum_{k=1}^{K}\sum_{n=1}^{N}z_{nk} * log(\frac{p_k}{p_k + w}^{x_n} * (1 - \frac{p_k}{p_k + w})^{1 - x_n})$

Найдем производную по $p$ и попытаемся найти экстремум функции:

3. $\frac{dl}{dp} = \frac{1}{p} \sum_{t=1}^{T}\sum_{n=1}^{N_t}z_{nt} x_n - \sum_{t=1}^{T}\sum_{n=1}^{N_t} \frac{z_{nt}}{p + w_t} = 0$

4. $\frac{dl}{dw} = \frac{1}{w}\sum_{k=1}^{K}\sum_{n=1}^{N}z_{nk} - \frac{1}{w}\sum_{k=1}^{K}\sum_{n=1}^{N}z_{nk}x_n - \sum_{k=1}^{K}\sum_{n=1}^{N}\frac{z_{nk}}{p_k + w} = 0$

Найти значения $p$ и $w$ сложно, поэтому я использовал scipy.optimize.fsolve

###### Отступление:
Изначально я посчитал формулы 3 и 4 неправильно как:

5. $\frac{dl}{dp} = \frac{1}{p} \sum_{t=1}^{T}\sum_{n=1}^{N_t}z_{nt} x_n - \frac{1}{p + w_t}\sum_{t=1}^{T}\sum_{n=1}^{N_t}z_{nt}  = 0$

6. $\frac{dl}{dw} = \frac{1}{w}\sum_{k=1}^{K}\sum_{n=1}^{N}z_{nk} - \frac{1}{w}\sum_{k=1}^{K}\sum_{n=1}^{N}z_{nk}x_n - \frac{1}{p_k + w}\sum_{k=1}^{K}\sum_{n=1}^{N}z_{nk} = 0$

И естественно легко максимизировал их и получил:

7. $p = \frac{\sum_{t=1}^{T}\sum_{n=1}^{N_t}z_{nt} x_n w_t}{\sum_{t=1}^{T}\sum_{n=1}^{N_t}z_{nt} - \sum_{t=1}^{T}\sum_{n=1}^{N_t}z_{nt} x_n}$


8. $w = \frac{\sum_{k=1}^{K}\sum_{n=1}^{N}z_{nk} p_k - \sum_{k=1}^{K}\sum_{n=1}^{N}z_{nk} p_k x_n}{\sum_{k=1}^{K}\sum_{n=1}^{N}z_{nk} x_n}$

По формуле 7 было видно, что чем больше отвеченных вопросов у знатока и чем больше сложность турнира, тем больше его сила и наоборот. А по формуле 8 было заметно, что чем больше неправильных ответов у сильного знатока, тем больше сложность вопроса, все вроде бы логично. НО при расчете этих формул была допущена грубая ошибка, которую я заметил только в последний день дедлайна, когда начал писать чистовую версию. Но эти значения все же использую как estimated value для scipy.optimize.fsolve

In [10]:
def dl_dmu(mu, z_x, znts, weights):
    """правдоподобие по формуле 5"""
    temp = z_x / mu
    for i, znt in enumerate(znts):
        temp -= znt / (weights[i] + mu)
    return temp

In [11]:
def dl_dw(weight, znk, z_x, mus):
    """правдоподобие по формуле 6"""
    result = (np.sum(znk) - np.sum(z_x)) / weight
    for mu, z in zip(mus, znk):
        result -= z / (weight + mu)
    return result

In [12]:
class EMModel:
    def __init__(self, train_ids, results, q=1.5, force=False, cut_num=None):
        self.z_vals = create_nested_dict(train_ids, results)
        self.pi_vals = create_nested_dict(train_ids, results)
        self.player_appereances = get_players_appereances(train_ids, results)
        self.players_mean_mu = create_players_dict(train_ids, results, True)
        # comp_probs = np.random.uniform(low=1.0, high=2.0, size=len(train_ids))
        comp_probs = np.ones(len(train_ids))
        self.comp_probs = {id_: prob for id_, prob in zip(train_ids, comp_probs)}
        
        self.train_ids = train_ids
        self.results = results
        self.q = q
        self.force = force
        self.cut_num = cut_num
        
    def calculate(self, func, eps=1e-5):
        for comp_id in tqdm.tqdm(self.comp_probs):
            weight = self.comp_probs[comp_id]
            for team in self.results[comp_id]:
                team_id = team['team']['id']
                mask = team['mask']
                mask = to_int(mask)
                func(team, mask, weight, comp_id, team_id, eps)
        
    def expectation(self, team, mask, weight, comp_id, team_id, eps):
        """Находим ожидание скрытых пременных"""
        if mask.shape[0] > 1 and len(team['teamMembers']) != 0:
            pis = []
            mus = []
            # Initialize
            for player in team['teamMembers']:
                player_id = player['player']['id']
                pis.append(self.pi_vals[comp_id][team_id][player_id])
                mus.append(self.players_mean_mu[player_id] / (self.q * self.players_mean_mu[player_id] + weight + eps))
                self.z_vals[comp_id][team_id][player_id] = []

            pis = np.array(pis)
            mus = np.array(mus)

            # Expectation
            for answer in mask:
                denominator = pis * (mus ** answer) * (1 - mus) ** (1 - answer)
                for i, player in enumerate(team['teamMembers']):
                    player_id = player['player']['id']
                    znk = denominator[i] / (np.sum(denominator) + eps)
                    assert znk <= 1, denominator
                    self.z_vals[comp_id][team_id][player_id].append(znk)

    def maxizmize_mus_and_pis_by_team(self, team, mask, weight, comp_id, team_id, eps):
        """Максимизируем pi по формуле 9"""
        if mask.shape[0] > 1 and len(team['teamMembers']) != 0:
            # Maximization
            for i, player in enumerate(team['teamMembers']):
                player_id = player['player']['id']
                nk = np.sum(self.z_vals[comp_id][team_id][player_id]) + eps
                new_mu = np.sum(mask * np.array(self.z_vals[comp_id][team_id][player_id]))
                if self.force:
                    k = np.log10(self.player_appereances[player_id])
                    k = pow(k, 0.25)
                    new_mu = new_mu * k
                self.z_x[player_id].append(new_mu)
                self.znt[player_id].append(nk)
                self.w_t[player_id].append(weight)
                self.pi_vals[comp_id][team_id][player_id] = nk / (mask.shape[0] + eps)

    def maximize_player_mus(self):
        """Максимизируем p по формуле 5"""
        for player_id in self.z_x:
            if self.cut_num:
                self.z_x[player_id] = self.z_x[player_id][-self.cut_num:]
                self.w_t[player_id] = self.w_t[player_id][-self.cut_num:]
                self.znt[player_id] = self.znt[player_id][-self.cut_num:]
                
            z_x = np.sum(self.z_x[player_id])
            mu = np.sum(np.array(self.z_x[player_id]) * np.array(self.w_t[player_id]))
            estimate_mu = mu / (np.sum(self.znt[player_id]) - np.sum(self.z_x[player_id]))
            new_mu = fsolve(dl_dmu, estimate_mu, args=(z_x, self.znt[player_id], self.w_t[player_id]), xtol=1)[0]
            self.players_mean_mu[player_id] = new_mu
            assert new_mu >= 0, (new_mu, mu, z_x, self.znt[player_id], self.w_t[player_id])

    def maxizmize_comp_weights(self, comp_id, eps=1e-5):
        """Максимизируем w по формуле 6"""
        znk_xn = []
        znk = []
        mus = []
        for team in self.results[comp_id]:
            team_id = team['team']['id']
            mask = team['mask']
            mask = to_int(mask)
            if mask.shape[0] > 1 and len(team['teamMembers']) != 0:
                # Maximization
                for i, player in enumerate(team['teamMembers']):
                    player_id = player['player']['id']
                    nk = np.sum(self.z_vals[comp_id][team_id][player_id]) + eps
                    summ = np.sum(mask * np.array(self.z_vals[comp_id][team_id][player_id]))
                    znk_xn.append(summ)
                    znk.append(nk)
                    mus.append(self.players_mean_mu[player_id])
        
        znk_uk_sum = np.sum(np.array(znk) * np.array(mus))
        znk_xn_uk_sum = np.sum(np.array(znk_xn) * np.array(mus))
        znk_xn_sum = np.sum(znk_xn)
        estimate_weight = (znk_uk_sum - znk_xn_uk_sum) / (znk_xn_sum + eps)
        self.comp_probs[comp_id] = fsolve(dl_dw, estimate_weight, args=(znk, znk_xn, mus), xtol=1)[0]
        assert self.comp_probs[comp_id] >= 0, self.comp_probs[comp_id]
            
    def maxizmize_comps_weights(self, eps=1e-5):
        for comp_id in self.comp_probs:
            self.maxizmize_comp_weights(comp_id)
            
    def iteration(self):
        self.z_x = create_players_dict(self.train_ids, self.results)
        self.znt = create_players_dict(self.train_ids, self.results)
        self.w_t = create_players_dict(self.train_ids, self.results)
        self.calculate(self.expectation)
        self.calculate(self.maxizmize_mus_and_pis_by_team)
        self.maximize_player_mus()
        self.maxizmize_comps_weights()

Обучим нашу модель:

In [13]:
model = EMModel(train_ids, results, q=1)
experiment_id = mlflow.set_experiment("EM-model")
with mlflow.start_run(experiment_id=experiment_id):
    for i in range(20):
        model.iteration()
        print('Iteration', i + 1)
        s_corr, k_corr = calc_correlations(test_ids, results, model.players_mean_mu)
        mlflow.log_metric("Spearman", s_corr, i + 1)
        mlflow.log_metric("Kendall", k_corr, i + 1)
        print('\n')

100%|██████████| 659/659 [00:08<00:00, 78.89it/s] 
/home/azamat/miniconda3/envs/conda_work/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in true_divide
  
/home/azamat/miniconda3/envs/conda_work/lib/python3.7/site-packages/scipy/optimize/minpack.py:163: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


Iteration 1
Spearman 0.7089191034986825
Kendall 0.5410043104935924




100%|██████████| 659/659 [00:08<00:00, 79.24it/s] 


Iteration 2
Spearman 0.7798524088098313
Kendall 0.6027060828760982




100%|██████████| 659/659 [00:08<00:00, 79.78it/s] 


...

Iteration 18
Spearman 0.8227425847606615
Kendall 0.6481458753377722




100%|██████████| 659/659 [00:08<00:00, 79.33it/s] 


Iteration 19
Spearman 0.8228246432490612
Kendall 0.648312418837321




100%|██████████| 659/659 [00:09<00:00, 73.07it/s] 


Iteration 20
Spearman 0.8229901368294043
Kendall 0.6483213660786957




По традиции посмотрим на топов:

In [14]:
top_players = get_top_players(model.players_mean_mu, 100, players, model.player_appereances)
for i, player in enumerate(top_players):
    print(i + 1, player[0], player[1], player[2])

1 Максим Руссо 7.7657410238455995 2178
2 Михаил Савченков 6.277050972799684 3215
3 Станислав Мереминский 6.261058006331137 1584
4 Максим Пилипенко 6.121525972355668 36
5 Евгений Спектор 6.095518092323639 230
6 Дмитрий Кудинов 6.040434132292189 45
7 Валентина Подюкова 6.019880624128774 36
8 Александра Брутер 5.763746766293936 2692
9 Иван Семушин 5.620082491448763 3774
10 Фёдор Иващенко 5.446749745222674 72
11 Артём Сорожкин 5.444043285029552 4849
12 Михаил Царёв 5.443063789828338 501
13 Анастасия Калинина 5.424305328786586 36
14 Михаил Левандовский 5.305090343351015 1457
15 Игорь Мокин 5.295188596876262 1176
16 Анна Карпелевич 5.1377195218976635 60
17 Антон Саксонов 5.116066863272906 1194
18 Алексей Гилёв 4.976092248080665 4306
19 Александр Либер 4.8340126852303245 3751
20 Мария Кленницкая 4.815174288221114 1172
21 Борис Моносов 4.791645105484871 1092
22 Ирина Прокофьева 4.788701999384437 1065
23 Андрей Островский 4.78335962459101 2612
24 Мария Юнгер 4.758621377442471 634
25 Илья Новико

А что там с вопросами? Постройте “рейтинг-лист” турниров по сложности вопросов.
Соответствует ли он интуиции (например, на чемпионате мира в целом должны быть
сложные вопросы, а на турнирах для школьников — простые)?

In [15]:
top_comps = get_top_comps(model.comp_probs, 100, tournaments)
for i, comp in enumerate(top_comps):
    print(i + 1, comp[0], comp[1])

1 Угрюмый Ёрш 6.067466387379861
2 Синхрон высшей лиги Москвы 5.129506271833155
3 Первенство правого полушария 4.5184465713253354
4 Воображаемый музей 4.286404746243713
5 Ускользающая сова 4.046374895576259
6 Записки охотника 3.8741600827607217
7 Знание – Сила VI 3.8715868549777217
8 Чемпионат Мира. Этап 2 Группа С 3.665938110251215
9 Чемпионат Минска. Лига А. Тур четвёртый 3.6505348787101433
10 Зеркало мемориала памяти Михаила Басса 3.6003141901704856
11 Чемпионат Мира. Этап 2. Группа В 3.4904738657355847
12 Кубок городов 3.4418577822027836
13 Чемпионат Мира. Этап 3. Группа С 3.4222977181679552
14 Чемпионат Мира. Финал. Группа С 3.4027519126421226
15 Чемпионат России 3.361487333550644
16 Чемпионат Мира. Этап 2. Группа А 3.3093066257712995
17 Чемпионат Мира. Этап 3. Группа В 3.2537479299407086
18 Чемпионат Мира. Этап 1. Группа С 3.1929324698358768
19 Тихий Донец: омут первый 3.153692096833067
20 All Cats Are Beautiful 3.129297414714623
21 VERSUS: Коробейников vs. Матвеев 3.1136917246619

## 6
Как мы видим в топе игроки, которые сыграли мало игр, попытаемся это исправить.

Для этого придумал 2 подхода:

1. В формулу 2 добавим параметр q > 1 (я назвал его параметром регуляризации) 

$p(x=1|k, t) = \frac{p_k}{q * p_k + w_t} = mu_{kt}$

Как мы видим этот параметр препятствует игроку стать слишком сильным, поэтому это должно сместить игроков ставших сильными засчет малого количества игр

2. Умножить силу игрока на функцию от количества сыгранных вопросов. Но это конечно явное притягивание за уши

In [ ]:
# Попробуем вариант 1
model = EMModel(train_ids, results, q=2)
experiment_id = mlflow.set_experiment("EM-model-with-q-regularization")
with mlflow.start_run(experiment_id=experiment_id):
    for i in range(20):
        model.iteration()
        print('Iteration', i + 1)
        s_corr, k_corr = calc_correlations(test_ids, results, model.players_mean_mu)
        mlflow.log_metric("Spearman", s_corr, i + 1)
        mlflow.log_metric("Kendall", k_corr, i + 1)
        print('\n')

In [ ]:
top_players = get_top_players(model.players_mean_mu, 100, players, model.player_appereances)
for i, player in enumerate(top_players):
    print(i + 1, player[0], player[1], player[2])

In [ ]:
top_comps = get_top_comps(model.comp_probs, 100, tournaments)
for i, comp in enumerate(top_comps):
    print(i + 1, comp[0], comp[1])

In [ ]:
# Попробуем вариант 2
model = EMModel(train_ids, results, q=2, force=True)
experiment_id = mlflow.set_experiment("EM-model-with-q-regularization-and-force")
with mlflow.start_run(experiment_id=experiment_id):
    for i in range(20):
        model.iteration()
        print('Iteration', i + 1)
        s_corr, k_corr = calc_correlations(test_ids, results, model.players_mean_mu)
        mlflow.log_metric("Spearman", s_corr, i + 1)
        mlflow.log_metric("Kendall", k_corr, i + 1)
        print('\n')

In [ ]:
top_players = get_top_players(model.players_mean_mu, 100, players, model.player_appereances)
for i, player in enumerate(top_players):
    print(i + 1, player[0], player[1], player[2])

In [ ]:
top_comps = get_top_comps(model.comp_probs, 100, tournaments)
for i, comp in enumerate(top_comps):
    print(i + 1, comp[0], comp[1])